In [142]:
### Extra code to reuse

## Exporting data to CSV
# combined_data.to_csv('file_name.csv')

## Finding what the Data Type a column is.
# data_type = IMDb_Basics['isAdult'].dtype
# print(data_type)

## Filtering dataframes for a specific values
# filtered_df = IMDB_Basics[IMDB_Basics['primaryTitle'].str.contains('Bond', case=False, na=False)]
# print(filtered_df)

# Optionally save the combined dataframe to a new CSV file
# output_file = path_to_files + 'combined_data.csv'
# combined_df.to_csv(output_file, index=False)
# print(f"Combined CSV saved to {output_file}")

# Code to troubleshoot date columns

# 1. Check for NaT values:
# If you used errors='coerce' during the conversion, you can check for NaT values, which would indicate conversion failures.
# nat_counts = all_stocks['Date'].isna().sum()
# print(f"Number of NaT values in 'Date': {nat_counts}")

# 2. Check the range of dates:
# Look at the minimum and maximum values to ensure they are within expected bounds.
# print("Earliest date:", all_stocks['Date'].min())
# print("Latest date:", all_stocks['Date'].max())

# 3. Sample random dates:
# Random sampling of dates can give you a quick spot check to see if dates look correctly formatted.
# print(all_stocks['Date'].sample(10))

# 4. Validate with known dates:
# If you know certain dates should exist in the dataset, check those specifically.
# known_dates = ["1997-05-15", "2005-07-08"]  # Example dates you expect to find
# for date in known_dates:
#     if pd.to_datetime(date) not in all_stocks['Date'].values:
#         print(f"Date {date} is not found or incorrectly formatted.")
#     else:
#         print(f"Date {date} is correctly formatted.")

# 5. Check for duplicates:
# Ensuring that there are no duplicate dates which could suggest an issue with the conversion.
# duplicate_dates = all_stocks['Date'].duplicated().sum()
# print(f"Number of duplicate dates: {duplicate_dates}")

## Remove duplicate rows based on the 'Release' column, keeping the first occurrence
# no_duplicates = no_duplicates.drop_duplicates(subset='Release', keep='first')

# # Remove the dollar sign and commas, then convert the column to numeric values
# no_duplicates['Gross'] = no_duplicates['Gross'].replace(r'[\$,]', '', regex=True).astype(float)
# # Calculate the mean of the 'Gross' column after conversion
# gross_mean = no_duplicates['Gross'].mean()
# # Now 'gross_mean' contains the average of the 'Gross' column as a float
# print(gross_mean)

## Drop a Single Column:
# df = df.drop('column_name', axis=1)

## Drop Multiple Columns:
# df = df.drop(['column1', 'column2'], axis=1)

Note: The gross revenue data is for US domestic only.

# First feedback from Abby

What I can look at is how did the preferences of the generes for movies changed before and after the pandemic.

I can look at x years previous of Covid and x years after Covid when the theatres were open. So drop all the movies that were launched during covid.

Then do the analysis.

# Project Structure for Analyzing Movie Genre Performance Before and After COVID-19:

### Objective:

Assess the financial performance of various movie genres before and after the onset of the COVID-19 pandemic.

### Data Collection:

Utilize the IMDb dataset to gather information on movies released in the years before and after the pandemic began (e.g., 2018-2019 for pre-COVID and 2020-2021 for post-COVID).
Focus on key data points like genre, box office earnings, budget, and release dates.
Analysis:

### Comparative Revenue Analysis: Compare the total and average box office revenues of different genres in both periods.
Genre Popularity Shifts: Identify any shifts in genre popularity. For example, did certain genres like horror or comedy become more popular post-COVID?
Budget vs. Revenue: Examine if there's a change in the budget-to-revenue ratio across genres before and after COVID-19.
Release Strategy Changes: Assess how release strategies (theatrical, streaming, hybrid) varied across genres and periods.

### Economic Theories and Models:

Apply microeconomic theories to explain changes in consumer preferences and market dynamics.
Consider demand and supply shifts due to external factors like lockdowns, economic uncertainty, and changes in consumer behavior.

### Results and Interpretation:

Present your findings in a clear, structured format.
Interpret the results in the context of economic theories and the unique circumstances of the pandemic.

### Conclusions:

Summarize key insights regarding the impact of COVID-19 on movie genre performance.
Discuss potential long-term implications for the film industry.

### Limitations and Further Research:

Acknowledge any limitations in your data or analysis.
Suggest areas for further research, such as a more detailed analysis of consumer preferences or the impact on independent films.
This project can offer a comprehensive view of how extraordinary events like a pandemic can influence entertainment industry trends, providing a practical application of economic concepts to real-world scenarios. Remember to keep your analysis aligned with the principles of microeconomics

Columns you should consider including in your analysis:

- Movie Title: The name of the movie. This is essential for identification and reference.

- Release Year: The year the movie was released. This will help you categorize movies into pre-COVID and post-COVID groups.

- Genre: The genre(s) of the movie. This is crucial for your primary analysis of comparing different genres.

- Box Office Revenue: The total earnings from ticket sales. This data is key to assessing financial performance.

- Budget: The estimated cost of producing the movie. This allows for analysis of profitability and budget-to-revenue ratios.

- Production Company: The company or companies that produced the movie. This can provide insights into market share and industry dynamics.

- Director and Key Cast Members: This information can be used to assess star power and its potential impact on a movie's success.

- IMDb Rating: Viewer ratings from IMDb. This can give a sense of critical reception and popularity.

- Number of Theaters Released In (if available): This data can help understand the scale of the release and its potential market reach.

- Streaming Platform Availability (post-COVID): For post-COVID movies, information on whether the movie was released on streaming platforms, and if so, which ones.

- Country/Countries of Origin: This helps in understanding geographical trends and the impact of regional COVID-19 measures.

- Runtime: The length of the movie, which can sometimes correlate with genre and audience preference.

- MPAA Rating/Censorship Rating: This information can provide insights into the target audience.

- Awards and Nominations: To gauge critical acclaim and its potential impact on financial success.

In [143]:
import pandas as pd
import os
import glob
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd, DateOffset

In [144]:
os.chdir('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5263 - Applied Microeconomics/Data Project/Data')

IMDb_title_basics = pd.read_csv('IMDb title basics.tsv', sep='\t', low_memory=False)
IMDb_Title_Principals = pd.read_csv('IMDb Title Principals.tsv', sep='\t', low_memory=False)
IMDb_title_ratings = pd.read_csv('IMDb title ratings.tsv', sep='\t', low_memory=False)
Name_Basics = pd.read_csv('Name Basics.tsv', sep='\t', low_memory=False)
# Title_Crew = pd.read_csv('Title Crew.tsv', sep='\t', low_memory=False)
# Title_Episode_Data = pd.read_csv('Title Episode Data.tsv', sep='\t', low_memory=False)
title_akas = pd.read_csv('title.akas.tsv', sep='\t', low_memory=False)

------------

Columns you should consider including in your analysis:

- <font color='Green'>Movie Title: From IMDb_title_basics - primaryTitle</font>

- Release Year: </font>

- <font color='Green'>Genre: From IMDb_title_basics - genres</font>

- Box Office Revenue: </font>

- Budget: </font>

- Production Company: </font>

- <font color='Green'>Director and Key Cast Members: category From IMDb_Title_Principals</font>

- <font color='Green'>IMDb Rating: averageRating - From IMDb_title_ratings</font>

- Number of Theaters Released In (if available): </font>

- Streaming Platform Availability (post-COVID): </font>

- Country/Countries of Origin: </font>

- <font color='Green'>Runtime: From IMDb_title_basics - runtimeMinutes</font>

- MPAA Rating/Censorship Rating: </font>

- Awards and Nominations: </font>

## Cleaning IMDb_title_basics

In [145]:
IMDb_title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


### All columns

- <font color='Green'>tconst: Keep </font>

- <font color='Green'>titleType: Keep</font>

- <font color='Green'>primaryTitle: Keep</font>

- <font color='Red'>originalTitle: Drop</font>

- <font color='Green'>isAdult: Keep</font>

- <font color='Red'>startYear: Drop</font>

- <font color='Red'>endYear: Drop</font>

- <font color='Green'>runtimeMinutes: Keep</font>

- <font color='Green'>genres: Keep</font>

In [146]:
# Dropping unnecessary columns
IMDb_title_basics = IMDb_title_basics.drop(['originalTitle', 'endYear', 'startYear'], axis=1)

# Filtering for movies only
IMDb_title_basics = IMDb_title_basics[IMDb_title_basics['titleType'] == 'movie']

In [147]:
# Creating isAdult as a dummy variable
IMDb_title_basics['isAdult'] = IMDb_title_basics['isAdult'].replace(0, 'No')

In [148]:
IMDb_title_basics.head()    # 81,487 rows  7 columns   1.2 MB

,tconst,titleType,primaryTitle,isAdult,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,0,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,0,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,0,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,0,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,0,90,Drama


--------

## Cleaning IMDb_Title_Principals

In [149]:
IMDb_Title_Principals.head()  # 3,911,728 rows  6 columns   1.1 GB

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


### All columns

- <font color='Green'>tconst: Keep</font>

- <font color='Green'>ordering: Keep</font>

- <font color='Green'>nconst: Keep</font>

- <font color='Green'>category: Keep</font>

- <font color='Red'>job: Drop</font>

- <font color='Red'>characters: Drop</font>

In [150]:
# Dropping unnecessary columns like job and characters
IMDb_Title_Principals = IMDb_Title_Principals.drop(['job', 'characters'], axis=1)

In [151]:
IMDb_Title_Principals.head()  # 3,911,728 rows  4 columns   1.1 GB

,tconst,ordering,nconst,category
0,tt0000001,1,nm1588970,self
1,tt0000001,2,nm0005690,director
2,tt0000001,3,nm0374658,cinematographer
3,tt0000002,1,nm0721526,director
4,tt0000002,2,nm1335271,composer


-----------

## Cleaning IMDb_title_ratings

In [152]:
IMDb_title_ratings.head()  # 1,058,804 rows  3 columns   27.5 MB   

,tconst,averageRating,numVotes
0,tt0000001,5.7,2007
1,tt0000002,5.8,270
2,tt0000003,6.5,1919
3,tt0000004,5.5,178
4,tt0000005,6.2,2696


----------

## Cleaning Name_Basics

In [153]:
Name_Basics.head()  # 9,007,366 rows  6 columns   1.1 GB

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0053137,tt0050419,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0075213,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0078723,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"


### All columns

- <font color='Green'>nconst: Keep</font>

- <font color='Green'>primaryName: Keep</font>

- <font color='Red'>birthYear: Drop</font>

- <font color='Red'>deathYear: Drop</font>

- <font color='Green'>primaryProfession: Keep</font>

- <font color='Green'>knownForTitles: Keep</font>

In [154]:
# dropping unnecessary columns like birthYear, deathYear
Name_Basics = Name_Basics.drop(['birthYear', 'deathYear'], axis=1)

In [155]:
Name_Basics.head()  # 9,007,366 rows  4 columns   1.1 GB

,nconst,primaryName,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,"soundtrack,actor,miscellaneous","tt0031983,tt0053137,tt0050419,tt0072308"
1,nm0000002,Lauren Bacall,"actress,soundtrack","tt0117057,tt0075213,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0056404,tt0054452"
3,nm0000004,John Belushi,"actor,soundtrack,writer","tt0080455,tt0078723,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"


----------

## Cleaning title_akas

In [156]:
title_akas.head()   # 30,000,000 rows  8 columns   2.5 GB

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


### All columns

- <font color='Green'>titleId: Keep</font>

- <font color='Green'>ordering: Keep</font>

- <font color='Red'>title: Drop</font>

- <font color='Green'>region: Keep</font>

- <font color='Green'>language: Keep</font>

- <font color='Green'>types	: Keep</font>

- <font color='Red'>attributes: Drop</font>

- <font color='Red'>isOriginalTitle: Drop</font>

In [157]:
# dropping unnecessary columns like title, isOriginalTitle and attributes
title_akas = title_akas.drop(['title', 'isOriginalTitle', 'attributes'], axis=1)

# rename the titleId column to tconst
title_akas = title_akas.rename(columns={'titleId': 'tconst'})

In [158]:
title_akas.head()   # 30,000,000 rows  5 columns   2.5 GB

,tconst,ordering,region,language,types
0,tt0000001,1,UA,\N,imdbDisplay
1,tt0000001,2,DE,\N,\N
2,tt0000001,3,HU,\N,imdbDisplay
3,tt0000001,4,GR,\N,imdbDisplay
4,tt0000001,5,RU,\N,imdbDisplay


---------

# merging 

In [159]:
# Merge title basics with ratings on 'tconst'
merged_df = pd.merge(IMDb_title_basics, IMDb_title_ratings[['tconst', 'averageRating']], on='tconst', how='left')

# Merge with akas to get 'region', then group by 'tconst' to combine regions into a list
regions_df = title_akas.groupby('tconst')['region'].apply(lambda x: ', '.join(x.dropna().astype(str).unique())).reset_index()

# Merge with regions
merged_df = pd.merge(merged_df, regions_df, on='tconst', how='left')



In [160]:
# Merge with principals to get 'nconst'
principals_df = pd.merge(merged_df, IMDb_Title_Principals[['tconst', 'nconst']], on='tconst', how='left')

# Merge principals with name basics to get 'primaryName' and 'primaryProfession'
# First, explode the 'knownForTitles' to be able to merge on 'tconst'
name_basics_exploded = Name_Basics.assign(knownForTitles=Name_Basics['knownForTitles'].str.split(',')).explode('knownForTitles')

# Then merge to get the names and professions associated with each 'nconst'
principals_with_names = pd.merge(principals_df, name_basics_exploded[['nconst', 'primaryName', 'primaryProfession', 'knownForTitles']], on='nconst', how='left')

# Filter for rows where 'primaryProfession' contains 'director'
directors_df = principals_with_names[principals_with_names['primaryProfession'].str.contains('director', na=False)]


In [161]:

# Group by 'tconst' and aggregate 'primaryName' into a list for director entries
director_names_df = directors_df.groupby('tconst')['primaryName'].apply(lambda x: ', '.join(x.dropna().astype(str).unique())).reset_index()

# Merge the director names with the directors dataframe
directors_df = pd.merge(directors_df, director_names_df, on='tconst', how='left', suffixes=('', '_directors'))

In [162]:
# Now select only the columns specified and rename 'primaryName_directors' back to 'primaryName'
directors_df = directors_df[['titleType', 'primaryTitle', 'genres', 'primaryName_directors', 'primaryProfession', 'averageRating', 'region', 'runtimeMinutes', 'isAdult']]
directors_df = directors_df.rename(columns={"primaryName_directors": "primaryName"})

In [163]:

# Group by the necessary columns and aggregate to combine regions into a list
final_df = directors_df.groupby(['titleType', 'primaryTitle', 'genres', 'primaryName', 'primaryProfession', 'averageRating', 'runtimeMinutes', 'isAdult']).agg({'region': lambda x: ', '.join(str(i) for i in x)}).reset_index()

# Drop duplicates and handle missing values as necessary
final_df = final_df.drop_duplicates()

final_df.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region
0,movie,!Women Art Revolution,Documentary,Lynn Hershman-Leeson,"director,producer,writer",6.8,83,0,"XWW, RU, GB, \N, US, XWW, RU, GB, \N, US, XWW,..."
1,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actor,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"
2,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actress,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"
3,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","producer,director,miscellaneous",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"
4,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","producer,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"


In [164]:
# filter duplicate values in the primaryTitle column
merged_IMDb_df = final_df.drop_duplicates(subset='primaryTitle', keep='first')

In [165]:
final_df.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region
0,movie,!Women Art Revolution,Documentary,Lynn Hershman-Leeson,"director,producer,writer",6.8,83,0,"XWW, RU, GB, \N, US, XWW, RU, GB, \N, US, XWW,..."
1,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actor,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"
2,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actress,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"
3,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","producer,director,miscellaneous",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"
4,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","producer,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US"


-------

In [ ]:
# filter merged_df primaryTitle for Avengers
avengers_df = final_df[final_df['primaryTitle'].str.contains('Avengers: Endgame', case=False, na=False)]
avengers_df.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region
59330,movie,Avengers: Endgame,"Action,Adventure,Drama","Anthony Russo, Joe Russo","producer,director,writer",8.4,181,0,"TW, AU, UA, US, SG, HR, KR, CL, EE, BG, IT, MX..."


In [ ]:
avengers_df.count()

titleType            1
primaryTitle         1
genres               1
primaryName          1
primaryProfession    1
averageRating        1
runtimeMinutes       1
isAdult              1
region               1
dtype: int64

In [ ]:
# # export avengers_df to csv
# avengers_df.to_csv('avengers_df.csv', index=False)

----------

<a name="_page0_x69.69_y423.22"></a>**IMDb Dataset Legend**

<Details>

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

<a name="_page0_x60.71_y555.04"></a>**title.akas.tsv.gz**

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title <a name="_page1_x60.71_y183.54"></a>**title.basics.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.002.png) endYear (YYYY) – TV Series end year. ‘\N’ for all other title types ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.003.png) runtimeMinutes – primary runtime of the title, in minutes ![](Aspose.Words.6ef9d709-0a24-4a63-90ff-a7a4701c44f5.004.png) genres (string array) – includes up to three genres associated with the title

<a name="_page1_x60.71_y549.05"></a>**title.crew.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- directors (array of nconsts) - director(s) of the given title
- writers (array of nconsts) – writer(s) of the given title

<a name="_page1_x60.71_y680.87"></a>**title.episode.tsv.gz**

- tconst (string) - alphanumeric identifier of episode
- parentTconst (string) - alphanumeric identifier of the parent TV Series
- seasonNumber (integer) – season number the episode belongs to
- episodeNumber (integer) – episode number of the tconst in the TV series

<a name="_page2_x60.71_y99.65"></a>**title.principals.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'
- characters (string) - the name of the character played if applicable, else '\N'

<a name="_page2_x60.71_y321.35"></a>**title.ratings.tsv.gz**

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

<a name="_page2_x60.71_y453.18"></a>**name.basics.tsv.gz**

- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for

# Revenues dataset below
---------

In [ ]:
# Define the path to the files
path_to_files = '/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5813 - Economic Analytics I/Project Proposal/Data/Gross Data/'

# Initialize an empty DataFrame for the Revenues_dataset
Revenues_dataset = pd.DataFrame()

# Define a helper function to extract the year from the filename
def get_year_from_filename(file_name):
    for month in ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]:
        if month in file_name:
            return file_name.split(month)[1].strip('.csv').strip()
    return None

# Loop through each CSV file in the directory
for file_name in os.listdir(path_to_files):
    if file_name.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(path_to_files, file_name)
        # Read the CSV file into a temporary DataFrame
        temp_df = pd.read_csv(file_path)
        # Extract the year from the file name
        year = get_year_from_filename(file_name)
        if year:
            # Assume that 'Release Date' column contains the day and abbreviated month (e.g., '17-Feb')
            # If day is missing, we'll prepend '01-' to use as a placeholder
            temp_df['Release Date'] = temp_df['Release Date'].str.extract(r'(\d+)-[A-Za-z]+', expand=False).fillna('01') + '-' + temp_df['Release Date'].str.extract(r'(\d+-)?([A-Za-z]+)', expand=False)[1] + '-' + year
            # Convert 'Release Date' to datetime format
            temp_df['Release Date'] = pd.to_datetime(temp_df['Release Date'], format='%d-%b-%Y', errors='coerce')
            # Drop rows where 'Release Date' could not be parsed
            temp_df.dropna(subset=['Release Date'], inplace=True)
            # Format 'Release Date' as 'MM/YYYY'
            temp_df['Release Date'] = temp_df['Release Date'].dt.strftime('%m/%Y')
            # Append to the main DataFrame
            Revenues_dataset = pd.concat([Revenues_dataset, temp_df], ignore_index=True)

# Remove rows with NaN 'Release Date' before dropping duplicates
Revenues_dataset.dropna(subset=['Release Date'], inplace=True)

# Drop duplicates based on the 'Release' column
Revenues_dataset = Revenues_dataset.drop_duplicates(subset='Release')

# Drop the unnecessary 'Rank' and 'Gross' columns
Revenues_dataset.drop(['Rank', 'Gross'], axis=1, inplace=True)

# Drop rows with null values in 'Total Gross'
Revenues_dataset = Revenues_dataset.dropna(subset=['Total Gross'])

Revenues_dataset.head()

,Release,Theaters,Total Gross,Release Date,Distributor
0,It Chapter Two,"4,570","$211,593,228",09/2019,Warner Bros.
1,Hustlers,"3,525","$104,963,598",09/2019,STX Entertainment
2,Downton Abbey,"3,548","$96,853,865",09/2019,Focus Features
3,Ad Astra,"3,460","$50,188,370",09/2019,Twentieth Century Fox
4,Rambo: Last Blood,"3,618","$44,819,352",09/2019,Lions Gate Films


In [ ]:
# filter merged_df primaryTitle for Avengers
avengers_df_31 = Revenues_dataset[Revenues_dataset['Release'].str.contains('Avengers: Endgame', case=False, na=False)]
avengers_df_31.head()

,Release,Theaters,Total Gross,Release Date,Distributor
90,Avengers: Endgame,"4,662","$858,373,000",04/2019,Walt Disney Studios Motion Pictures


In [ ]:
final_df.tail()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region
728833,movie,à propos: philosophie,Documentary,"Sebastian Köthe, Roman Hagenbrock","editor,director,cinematographer",8.5,80,0,"\N, DE, \N, DE, \N, DE"
728834,movie,à propos: philosophie,Documentary,"Sebastian Köthe, Roman Hagenbrock","writer,director,producer",8.5,80,0,"\N, DE, \N, DE, \N, DE, \N, DE"
728835,movie,ärtico,Drama,"Gabriel Velázquez, Manuel García, Blanca Torres","director,writer,producer",5.6,78,0,"\N, FR, GR, RU, ES, SE, DE, \N, FR, GR, RU, ES..."
728836,movie,ärtico,Drama,"Gabriel Velázquez, Manuel García, Blanca Torres","writer,cinematographer,assistant_director",5.6,78,0,"\N, FR, GR, RU, ES, SE, DE, \N, FR, GR, RU, ES..."
728837,movie,ärtico,Drama,"Gabriel Velázquez, Manuel García, Blanca Torres","writer,editor,director",5.6,78,0,"\N, FR, GR, RU, ES, SE, DE, \N, FR, GR, RU, ES..."


In [ ]:
# merge final_df's ciolumn called primaryTitle with revenues dataset's column called Release
combined_data = pd.merge(final_df, Revenues_dataset[['Release', 'Total Gross']], left_on='primaryTitle', right_on='Release', how='left')
combined_data.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region,Release,Total Gross
0,movie,!Women Art Revolution,Documentary,Lynn Hershman-Leeson,"director,producer,writer",6.8,83,0,"XWW, RU, GB, \N, US, XWW, RU, GB, \N, US, XWW,...",NaN,NaN
1,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actor,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN
2,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actress,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN
3,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","producer,director,miscellaneous",5.7,87,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN
4,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","producer,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN


In [ ]:
# delete duplicate values in primaryTitle column
combined_data = combined_data.drop_duplicates(subset='primaryTitle', keep='first')
combined_data.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region,Release,Total Gross
0,movie,!Women Art Revolution,Documentary,Lynn Hershman-Leeson,"director,producer,writer",6.8,83,0,"XWW, RU, GB, \N, US, XWW, RU, GB, \N, US, XWW,...",NaN,NaN
1,movie,#1 Serial Killer,Horror,"Jason Tobin, Shoshana Bush, Stanley Yung, Quen...","actor,director,writer",5.7,87,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN
5,movie,#5,"Biography,Comedy,Fantasy",Ricky Bardy,"producer,director,editor",5.0,68,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN
6,movie,#50Fathers,Comedy,"Joddy Eric Matthews, Denny Castiglione, Jay B....","actor,producer,assistant_director",4.3,132,0,"\N, US, \N, US, \N, US, \N, US",NaN,NaN
10,movie,#66,Action,Asun Mawardi,"producer,director,writer",5.6,116,0,"GB, ID, \N, GB, ID, \N, GB, ID, \N, GB, ID, \N",NaN,NaN


In [ ]:
# delete all null values in Release and total gross columns
combined_data = combined_data.dropna(subset=['Release', 'Total Gross'])


In [ ]:
combined_data.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region,Release,Total Gross
322,movie,'71,"Action,Crime,Drama","Sean Harris, Yann Demange, Robin Gutch","actor,writer,director",7.2,99,0,"UY, CZ, NL, FI, GB, AU, CN, IN, ES, XWW, DK, \...",'71,"$1,270,847"
331,movie,'83,"Biography,Drama,History","Kabir Khan, Sumit Arora, Vasan Bala, Sanjay Pu...","director,writer,actor",7.5,162,0,"AR, \N, NZ, TR, IN, CN, CA, AE, PT, FR, ID, XW...",'83,"$4,055,660"
335,movie,'85: The Greatest Team in Football History,"Documentary,Sport",Scott Prestin,"producer,director,actor",6.8,90,0,"\N, GB, US, \N, GB, US, \N, GB, US, \N, GB, US",'85: The Greatest Team in Football History,"$124,573"
1184,movie,"1,000 Times Good Night","Drama,War","Erik Poppe, Kirsten Sheridan","director,producer,cinematographer",7.0,117,0,"JP, MX, HR, SE, VE, TR, AR, ES, BG, CL, \N, FI...","1,000 Times Good Night","$53,895"
1257,movie,10 Cloverfield Lane,"Drama,Horror,Mystery","Dan Trachtenberg, Damien Chazelle","music_department,director,writer",7.2,103,0,"SG, RO, CN, SI, CL, GR, SE, PT, VN, BG, UY, GB...",10 Cloverfield Lane,"$72,082,998"


In [ ]:
combined_data.count()

titleType            5682
primaryTitle         5682
genres               5682
primaryName          5682
primaryProfession    5682
averageRating        5682
runtimeMinutes       5682
isAdult              5682
region               5682
Release              5682
Total Gross          5682
dtype: int64

In [ ]:
# filter merged_df primaryTitle for Avengers
avengers_df_1 = combined_data[combined_data['primaryTitle'].str.contains('Avengers: Endgame', case=False, na=False)]
avengers_df_1.head()

,titleType,primaryTitle,genres,primaryName,primaryProfession,averageRating,runtimeMinutes,isAdult,region,Release,Total Gross
59330,movie,Avengers: Endgame,"Action,Adventure,Drama","Anthony Russo, Joe Russo","producer,director,writer",8.4,181,0,"TW, AU, UA, US, SG, HR, KR, CL, EE, BG, IT, MX...",Avengers: Endgame,"$858,373,000"


# Loading new Budget data

In [ ]:
os.chdir('/Users/shreyashgupta/Library/CloudStorage/OneDrive-UniversityofArkansas/MSEA/ECON 5263 - Applied Microeconomics/Data Project/Data/Box Office Mojo Dataset')

Mojo_budget_data_1 = pd.read_csv('Mojo_budget_data.csv')
Mojo_budget_data_2 = pd.read_csv('Mojo_budget_update.csv')

In [ ]:
# Add merge mojo_budget_data_1 below mojo_budget_data_2
Mojo_budget_data = pd.concat([Mojo_budget_data_1, Mojo_budget_data_2], ignore_index=True)

Mojo_budget_data.head()

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,genre_2,genre_3,genre_4,link,title,year,trivia,release_date,distributor,html
0,tt0118589,Glitter,2001.0,Vondie Curtis-Hall,Cheryl L. West,Laurence Mark,Terence Blanchard,Geoffrey Simpson,Mariah Carey,Eric Benét,...,Music,Romance,NaN,https://www.boxofficemojo.com/title/tt0118589/...,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0120630,Chicken Run,2000.0,Peter Lord,Peter Lord,Peter Lord,Harry Gregson-Williams,Simon Jacobs,Mel Gibson,Julia Sawalha,...,Animation,Comedy,Drama,https://www.boxofficemojo.com/title/tt0120630/...,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0120667,Fantastic Four,2005.0,Tim Story,Mark Frost,Avi Arad,John Ottman,Oliver Wood,Ioan Gruffudd,Michael Chiklis,...,Adventure,Family,Fantasy,https://www.boxofficemojo.com/title/tt0120667/...,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0120679,Frida,2002.0,Julie Taymor,Hayden Herrera,Lindsay Flickinger,Elliot Goldenthal,Rodrigo Prieto,Salma Hayek,Alfred Molina,...,Drama,Romance,NaN,https://www.boxofficemojo.com/title/tt0120679/...,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0120681,From Hell,2001.0,Albert Hughes,Alan Moore,Jane Hamsher,Trevor Jones,Peter Deming,Johnny Depp,Heather Graham,...,Mystery,Thriller,NaN,https://www.boxofficemojo.com/title/tt0120681/...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Mojo_budget_data.count()

movie_id           5719
movie_title        2476
movie_year         2476
director           5719
writer             5706
producer           5701
composer           5541
cinematographer    5515
main_actor_1       5719
main_actor_2       5719
main_actor_3       5719
main_actor_4       5716
budget             5719
domestic           5700
international      5309
worldwide          5712
mpaa               5496
run_time           5719
genre_1            5719
genre_2            5257
genre_3            3951
genre_4            2026
link               2476
title              3243
year               3243
trivia             3243
release_date       3242
distributor        3228
html               3243
dtype: int64

In [ ]:
# filter Mojo_budget_data primaryTitle for Avengers: Endgame
avengers_df_2 = Mojo_budget_data[Mojo_budget_data['movie_title'].str.contains('Avengers: Endgame', case=False, na=False)]
avengers_df_2.head()

,movie_id,movie_title,movie_year,director,writer,producer,composer,cinematographer,main_actor_1,main_actor_2,...,genre_2,genre_3,genre_4,link,title,year,trivia,release_date,distributor,html
2285,tt4154796,Avengers: Endgame,2019.0,Anthony Russo,Christopher Markus,Kevin Feige,Alan Silvestri,Trent Opaloch,Robert Downey Jr.,Chris Evans,...,Adventure,Drama,Sci-Fi,https://www.boxofficemojo.com/title/tt4154796/...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
avengers_df_2.columns

Index(['movie_id', 'movie_title', 'movie_year', 'director', 'writer',
       'producer', 'composer', 'cinematographer', 'main_actor_1',
       'main_actor_2', 'main_actor_3', 'main_actor_4', 'budget', 'domestic',
       'international', 'worldwide', 'mpaa', 'run_time', 'genre_1', 'genre_2',
       'genre_3', 'genre_4', 'link', 'title', 'year', 'trivia', 'release_date',
       'distributor', 'html'],
      dtype='object')